# 🧠 Mental Health in Students Post-COVID: A Silent Crisis?
#### 📊 Preprocessing Notebook + Exploratory Data Analysis (EDA)


> *"The classrooms reopened, but something never quite returned."*  
> *"Smiles came back, but the silence grew louder."*

The COVID-19 pandemic reshaped the world in ways we are still trying to understand—and among its most silent victims were **students**. Behind the return to academic routine lies a shadow crisis: **increased anxiety, depression, sleep disorders, and even suicidal thoughts**. This project aims to explore and analyze the **mental health impact on students post-COVID**, using real-world data that includes psychological, academic, lifestyle, and demographic variables.

Through this notebook, we will:
- 🧹 Perform **data preprocessing** to clean and shape the dataset.
- 📊 Conduct **exploratory data analysis (EDA)** to uncover patterns, correlations, and warning signs.
- 🎯 Use visualizations that not only highlight trends, but **tell the story of students' struggles** in a post-pandemic world.

This is not just a statistical exercise — it's an attempt to quantify a **crisis hidden in plain sight**.

Let’s begin.

## 🔧 Imports
Below are the necessary libraries required for data handling, visualization, and basic preprocessing.


In [136]:
# 📦 Basic Imports

import pandas as pd

# 📊 Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# 🛠️ Display Settings
plt.style.use('ggplot')
sns.set_palette('Set2')
import warnings
warnings.filterwarnings('ignore')

# 🧠 Optional: For interactive visuals (if you plan to add later)
# import plotly.express as px
# import cufflinks as cf
# cf.go_offline()




## 📂Loading and Combining Datasets

We are working with **two publicly available datasets** focused on student mental health after pandemic.  
Both datasets have nearly identical structures, but we ensured consistency by applying minor modifications before merging them into a single dataframe.

> Final merged dataset will be the foundation of all our preprocessing and analysis.


In [85]:
# Load the first cleaned dataset (already preprocessed and saved)
df1 = pd.read_excel(r'datasets/final_depression_dataset_2.xlsx')

# Load the second dataset which had original raw data
df2 = pd.read_excel(r'datasets/student_depression_dataset.xlsx')

Modifications of both datasets before merging

In [86]:
#Drop unnecessary columns from both datasets
# These columns are not needed for our analysis and visualization
df1 = df1.drop(columns=['Profession', 'Work Pressure', 'Job Satisfaction'])
df2 = df2.drop(columns=['Profession', 'Work Pressure', 'Job Satisfaction'])

In [87]:
#Renaming columns for consistency
# This ensures both datasets have the same column names for merging
df1 = df1.rename(columns={'Have you ever had suicidal thoughts ?': 'Suicidal Thoughts',
                        'Work/Study Hours': 'Study Hours',})
df2 = df2.rename(columns={'Have you ever had suicidal thoughts ?': 'Suicidal Thoughts',
                        'Work/Study Hours': 'Study Hours',})

Merge both DataFrames

In [88]:
#Merged both DataFrames using pd.concat
# This combines the two datasets into a single DataFrame for analysis
merged_df = pd.concat([df1, df2], ignore_index=True)

You can view your new Dataframe information 

In [ ]:
merged_df.info()

We will see how our new DataFrame is looking 

In [ ]:
merged_df.head(5)

# 🧹 Data Preprocessing & Normalization

Before we dive into exploratory analysis, we will perform essential preprocessing steps and normalize categorical column values to ensure consistency and enhance the clarity of our visualizations.


Convert Male/Female to M/F

In [91]:
#Using map function to convert Male/Female to M/F
# This creates cleaner labels for plots and improves readability in visual analysis
merged_df['Gender'] = merged_df['Gender'].map({'Male' : 'M' , 'Female':'F'})

#### Convert All columns who has Yes/No to 1/0
Converting to 1/0 simplifies analysis and makes the data ready for statistical and machine learning models.

In [92]:
#We will create a list of columns that have Yes/No values
# Converting these columns to 1/0 for easier analysis
lst = ['Suicidal Thoughts' , 'Family History of Mental Illness' , 'Depression']

#Using loop to iterate through the list and map Yes/No to 1/0
# This simplifies the data for statistical analysis and visualization
for columns in lst:
    merged_df[columns] = merged_df[columns].map({'Yes': '1', 'No': '0'})

In [ ]:
merged_df['Sleep Duration'].value_counts()
#You can see here due to comma we have repeated values in different formats
#We will remove those commas to make it consistent
#Then we will convert the column to numeric type for better analysis

Convert Sleep Duration into numeric

In [94]:
#we will create a mapping dictionary to convert sleep duration strings to numeric values
mapping = {
    '5-6 hours': 5.5,
    "'5-6 hours'": 5.5,
    '7-8 hours': 7.5,
    "'7-8 hours'": 7.5,
    'More than 8 hours': 9,
    "'More than 8 hours'": 9,
    'Less than 5 hours': 4.5,
    "'Less than 5 hours'": 4.5,
    'Others': np.nan  # You can impute or drop later
}

#By using the map function and mapping dict, we can convert the 'Sleep Duration' column to numeric values
merged_df['Sleep Duration'] = merged_df['Sleep Duration'].map(mapping)

We have 18 Null Values due to Other 

In [ ]:
merged_df['Sleep Duration'].isnull().sum()

In [96]:
#Using fillna to handle missing values in 'Sleep Duration'
# This replaces NaN values with the mean of the column for better analysis
merged_df['Sleep Duration'].fillna(merged_df['Sleep Duration'].mean(), inplace=True)

In [ ]:
#Now you can check the number of null values in 'Sleep Duration' again
merged_df['Sleep Duration'].isnull().sum()

Convert Healthy, Moderate and Unhealthy into Numeric values for better analysis

In [98]:
#Convert Healthy, Moderate and Unhealthy into Numeric values for better analysis
# This will help in statistical analysis and visualization
merged_df['Dietary Habits'] = merged_df['Dietary Habits'].map({'Healthy':'2' , 'Unhealthy':'0' , 'Moderate':'1'})

Delete Rows who's Age is Greater than 27 for focusing on Student Age

In [105]:
merged_df = merged_df[merged_df['Age'] < 28]

Removing Invalid City Rows

In [ ]:
#When you see there are some rows who doesn't have a valid city name
# We will remove those rows to focus on valid city data
merged_df['City'].value_counts()

In [ ]:

# Remove rows where the city count is less than 20
city_counts = merged_df['City'].value_counts()
#This will filter out cities that have less than 20 entries
valid_cities = city_counts[city_counts >= 20].index
# Filter the DataFrame to keep only rows with valid cities
merged_df = merged_df[merged_df['City'].isin(valid_cities)]


There is Class 12 and 'Class 12' So merge them in one

In [189]:
# Strip leading/trailing spaces and quotes from Degree column
merged_df['Degree'] = merged_df['Degree'].str.strip().str.replace("'", "")


Save Cleaned DataFrame into a Excel File

In [193]:
merged_df.to_excel(r'post_covid_mental_health_cleaned.xlsx', index=False)

# 🧠 Exploratory Data Analysis (EDA): Understanding the Silent Crisis

The numbers alone don’t speak — we make them speak.

In this section, we dive deep into the mental health landscape of students post-COVID, not just through cold data points but by uncovering patterns, red flags, and subtle signals hidden within. 📊  
With each plot, we attempt to decode **what the students aren’t saying out loud** depression, suicidal thoughts all silently impacting lives behind good grades and long study hours.

This isn’t just data analysis it’s a step toward **recognizing a crisis** that’s often dismissed. Let's visualize the truth, one plot at a time.

Create a DataFrame of our Cleaned File

In [77]:
df = pd.read_excel(r'post_covid_mental_health_cleaned.xlsx')

#### 🎯 Categorical Overview: Who’s Represented in This Data?

Before diving into deeper correlations, let's look at the basic composition of our dataset.  
These countplots help us visualize the distribution of students across depression , gender, cities and more along with how many of them reported depression or suicidal thoughts. 📊  


In [ ]:
#Basic overview of the DataFrame through countplots
# 0 -> No Depression, 1 -> Depression
# 0 -> No Suicidal Thoughts, 1 -> Suicidal Thoughts
fig, axes = plt.subplots(2, 2, figsize=(15,10))

# ...existing code...
sns.countplot(data=df, x='Depression', ax=axes[0,0])
axes[0,0].set_title('Depression Count')
for p in axes[0,0].patches:
    axes[0,0].annotate(
        int(p.get_height()),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha='center', va='bottom', fontsize=10, fontweight='bold', color='black'
    )

sns.countplot(data=df, x='Suicidal Thoughts', ax=axes[0,1])
axes[0,1].set_title('Suicidal Thoughts Count')
for p in axes[0,1].patches:
    axes[0,1].annotate(
        int(p.get_height()),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha='center', va='bottom', fontsize=10, fontweight='bold', color='black'
    )

sns.countplot(data=df, x='Gender', ax=axes[1,0])
axes[1,0].set_title('Gender Count')
for p in axes[1,0].patches:
    axes[1,0].annotate(
        int(p.get_height()),
        (p.get_x() + p.get_width() / 2, p.get_height()),
        ha='center', va='bottom', fontsize=10, fontweight='bold',)

sns.countplot(data=df, y='Degree', ax=axes[1,1])
axes[1,1].set_title('Degree Count')

plt.tight_layout()
plt.show()


### 💡Gender-wise Distribution of Depression and Suicidal Thoughts

Are male and female students equally affected by depression and suicidal thoughts?  
Let’s visualize and compare the mental health condition distribution across gender.  


In [ ]:
plt.figure(figsize=(12,5))

# Calculate percentages for Depression by Gender
depr_counts = df.groupby(['Gender', 'Depression']).size().unstack(fill_value=0)
depr_percent = (depr_counts[1] / depr_counts.sum(axis=1) * 100).round(1)

# Depression vs Gender
plt.subplot(1, 2, 1)
sns.countplot(data=df, x='Gender', hue='Depression', palette='coolwarm')
plt.title(f"Depression: {depr_percent.get('M',0)}% Males, {depr_percent.get('F',0)}% Females Depressed")
plt.xlabel('Gender')
plt.ylabel('Count')

# Calculate percentages for Suicidal Thoughts by Gender
suic_counts = df.groupby(['Gender', 'Suicidal Thoughts']).size().unstack(fill_value=0)
suic_percent = (suic_counts[1] / suic_counts.sum(axis=1) * 100).round(1)

# Suicidal Thoughts vs Gender
plt.subplot(1, 2, 2)
sns.countplot(data=df, x='Gender', hue='Suicidal Thoughts', palette='Set2')
plt.title(f"Suicidal Thoughts: {suic_percent.get('M',0)}% Males, {suic_percent.get('F',0)}% Females")
plt.xlabel('Gender')
plt.ylabel('Count')

plt.tight_layout()
plt.show()


### 🎯 Age vs Depression & Suicidal Thoughts

Mental health issues don’t affect all age groups equally. Some students are at a much higher risk than others due to developmental, academic, or social pressures.

Let’s explore which age group is most vulnerable to **depression** and **suicidal thoughts** among students.  
This analysis helps us **identify the most at-risk group**, enabling focused mental health interventions. 🧠⚠️

> 🔍 The age group with the **highest depression cases** is highlighted in the chart title below.


In [ ]:
plt.figure(figsize=(12,6))


most_deprr_age = df.groupby(['Age'])['Depression'].value_counts().unstack(fill_value=0)
        

max_age = most_deprr_age[1].idxmax()
max_count = most_deprr_age[1].max()


print(f"Students of Age:{max_age}, why are you all so depressed? 😭 But hey, we’re here for you sending hugs 🫂")

plt.subplot(1,2,1)
sns.countplot(data=df , x='Age', hue='Depression')


plt.subplot(1,2,2)
sns.countplot(data=df , x='Age', hue='Suicidal Thoughts', palette='Set2')

plt.suptitle(f" Age {max_age} Has Highest Depression and Suicidal Thoughts ({max_count} students)", fontsize=14)
plt.tight_layout()
plt.show()





### 🎓 Which Degrees Face the Most Depression and Suicidal Thoughts?

Not all academic paths carry the same mental load.  
Let’s find out which degree programs have the **highest percentage** of students reporting depression and suicidal thoughts. 📊

The chart below reveals the **Top 20 degrees** where students are most affected.  
This can help institutions focus mental health resources where they’re needed most. ⚠️🧠


In [ ]:
# Step 1: Calculate % of depressed students in each degree
degree_depression_pct = (
    df[df['Depression'] == 1]
    .groupby('Degree')
    .size() / df.groupby('Degree').size()
) * 100

# Step 2: Drop NaNs (if any) and sort
degree_depression_pct = degree_depression_pct.dropna().sort_values(ascending=False)

# Step 3: Take top 20
top_20_pct = degree_depression_pct.head(20)

# Step 4: Plot with labels
plt.figure(figsize=(10, 8))
barplot = sns.barplot(x=top_20_pct.values, y=top_20_pct.index, palette='flare')

# Add % labels on bars
for i, value in enumerate(top_20_pct.values):
    plt.text(value + 1, i, f"{value:.1f}%", va='center', fontsize=9)

plt.title('Top 20 Degrees with Highest % of Depression 🧠💔', fontsize=14)
plt.xlabel('Depression Percentage (%)')
plt.ylabel('Degree')
plt.xlim(0, 100)
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Calculate % of depressed students in each degree
degree_depression_pct = (
    df[df['Suicidal Thoughts'] == 1]
    .groupby('Degree')
    .size() / df.groupby('Degree').size()
) * 100

# Step 2: Drop NaNs (if any) and sort
degree_depression_pct = degree_depression_pct.dropna().sort_values(ascending=False)

# Step 3: Take top 20
top_20_pct = degree_depression_pct.head(20)

# Step 4: Plot with labels
plt.figure(figsize=(10, 8))
barplot = sns.barplot(x=top_20_pct.values, y=top_20_pct.index, palette='flare')

# Add % labels on bars
for i, value in enumerate(top_20_pct.values):
    plt.text(value + 1, i, f"{value:.1f}%", va='center', fontsize=9)

plt.title('Top 20 Degrees with Highest % of Suicidal Thoughts 🧠💔', fontsize=14)
plt.xlabel('Depression Percentage (%)')
plt.ylabel('Degree')
plt.xlim(0, 100)
plt.tight_layout()
plt.show()


In [ ]:
# Filter out Class 12
df_filtered = df[df['Degree'] != 'Class 12']

# Calculate depressed counts
depression_counts = (
    df_filtered[df_filtered['Depression'] == 1]
    .groupby(['Degree','Gender'])
    .size()
    .reset_index(name='Depressed_Count')
)

# Sort separately for boys and girls
boys_order = (
    depression_counts[depression_counts['Gender']=='M']
    .sort_values(by='Depressed_Count', ascending=False)['Degree']
    .drop_duplicates()
    .head(10)
)
girls_order = (
    depression_counts[depression_counts['Gender']=='F']
    .sort_values(by='Depressed_Count', ascending=False)['Degree']
    .drop_duplicates()
    .head(10)
)

# Boys plot (only depressed students for clarity)
boys_df = df_filtered[(df_filtered['Gender']=='M') & (df_filtered['Depression']==1)]
# --- Boys Plot with Counts ---
plt.figure(figsize=(10,6))
ax = sns.countplot(
    data=boys_df,
    y='Degree',
    order=boys_order,
    palette='mako'
)
plt.title("Top 10 Degrees with Highest Depressed Boys (Excluding Class 12)")
plt.xlabel("Depressed Students Count")
plt.ylabel("Degree")

# Add count labels on bars
for p in ax.patches:
    width = p.get_width()  # bar length
    plt.text(width + 0.3,        # X position (slightly right of bar)
             p.get_y() + p.get_height()/2,  # Y center of bar
             int(width),         # Label = count
             va='center')

plt.tight_layout()
plt.show()


# Girls plot (only depressed students)
girls_df = df_filtered[(df_filtered['Gender']=='F') & (df_filtered['Depression']==1)]
plt.figure(figsize=(10,6))

ax = sns.countplot(
    data=girls_df,
    y='Degree',
    order=girls_order,
    palette = 'viridis'
)
plt.title("Top 10 Degrees with Highest Depressed Girls (Excluding Class 12)")
plt.xlabel("Depressed Students Count")
plt.ylabel("Degree")
# Add count labels on bars
for p in ax.patches:
    width = p.get_width()
    plt.text(width + 0.3,        # X position (slightly right of bar)
             p.get_y() + p.get_height()/2,  # Y center of bar
             int(width),         # Label = count
             va='center')
plt.tight_layout()
plt.show()

# Highest depressed degree for each gender
highest_boys = (
    depression_counts[depression_counts['Gender']=='M']
    .sort_values(by='Depressed_Count', ascending=False)
    .iloc[0]
)
highest_girls = (
    depression_counts[depression_counts['Gender']=='F']
    .sort_values(by='Depressed_Count', ascending=False)
    .iloc[0]
)

print(f"In Boys, {highest_boys['Degree']} has the most depressed students ({highest_boys['Depressed_Count']}) 💀")
print(f"In Girls, {highest_girls['Degree']} has the most depressed students ({highest_girls['Depressed_Count']}) 💀")


In [ ]:

# Convert Sleep to categorical & sort by average depression rate
sleep_df = df.copy()
sleep_summary = (
    sleep_df.groupby('Sleep Duration')['Depression']
    .agg(['mean','sum','count'])
    .sort_values(by='sum', ascending=False)
)
sleep_df['Sleep Duration'] = pd.Categorical(
    sleep_df['Sleep Duration'], 
    categories=sleep_summary.index, 
    ordered=True
)

plt.figure(figsize=(12,6))
ax = sns.countplot(
    data=sleep_df,
    x='Sleep Duration',
    hue='Depression',
    palette=["#57dd3c","#ee4c4c"]
)

# Add dynamic count labels
for p in ax.patches:
    height = p.get_height()
    x_pos = p.get_x() + p.get_width()/2
    ax.text(x_pos, height + 1, int(height), ha='center', va='bottom', fontsize=10)

# Find worst sleep dynamically
worst_sleep = sleep_summary['mean'].idxmax()
worst_rate = round(sleep_summary['mean'].max()*100,1)

# Dynamic title
plt.title(
    f"😴 Sleep vs Depression\n"
    f"Students sleeping {worst_sleep} hrs or less have the highest depression rate ({worst_rate}%)", 
    fontsize=14, fontweight='bold'
)
plt.xlabel("Sleep Duration (Hours)")
plt.ylabel("Number of Students")
plt.legend(title='Depression', labels=['No', 'Yes'])

# Annotate dynamically on worst bar
worst_bar = ax.patches[list(sleep_summary.index).index(worst_sleep)*2+1]  # depressed bar
plt.annotate(
    f"⚠ {worst_rate}% depressed here!",
    xy=(worst_bar.get_x()+worst_bar.get_width()/2, worst_bar.get_height()),
    xytext=(0, worst_bar.get_height()+1000),
    ha='center',
    arrowprops=None,
    fontsize=11, color="#000000"
)

plt.tight_layout()
plt.show()

# Dynamic Summary
print(f"📊 Insight: Students sleeping {worst_sleep} hrs show the highest depression rate ({worst_rate}%). "
      f"Longer sleep is associated with lower depression risk.")


In [78]:
df['Sleep Duration'] = df['Sleep Duration'].replace(6.490558393517703,6.5)

In [ ]:


# Convert Sleep to categorical & sort by average depression rate
sleep_df = df.copy()
sleep_summary = (
    sleep_df.groupby('Sleep Duration')['Suicidal Thoughts']
    .agg(['mean','sum','count'])
    .sort_values(by='sum', ascending=False)
)
sleep_df['Sleep Duration'] = pd.Categorical(
    sleep_df['Sleep Duration'], 
    categories=sleep_summary.index, 
    ordered=True
)

plt.figure(figsize=(12,6))
ax = sns.countplot(
    data=sleep_df,
    x='Sleep Duration',
    hue='Suicidal Thoughts',
    palette=["#86d0e3","#c658b6"]
)

# Add dynamic count labels
for p in ax.patches:
    height = p.get_height()
    x_pos = p.get_x() + p.get_width()/2
    ax.text(x_pos, height + 1, int(height), ha='center', va='bottom', fontsize=10)

# Find worst sleep dynamically
worst_sleep = sleep_summary['mean'].idxmax()
worst_rate = round(sleep_summary['mean'].max()*100,1)

# Dynamic title
plt.title(
    f"😴 Sleep vs Suicidal Thoughts\n"
    f"Students sleeping {worst_sleep} hrs or less have the highest suicidal thoughts rate ({worst_rate}%)", 
    fontsize=14, fontweight='bold'
)
plt.xlabel("Sleep Duration (Hours)")
plt.ylabel("Number of Students")
plt.legend(title='Suicidal Thoughts', labels=['No', 'Yes'])

# Annotate dynamically on worst bar
worst_bar = ax.patches[list(sleep_summary.index).index(worst_sleep)*2+1]  # depressed bar
plt.annotate(
    f"⚠ {worst_rate}% here!",
    xy=(worst_bar.get_x()+worst_bar.get_width()/2, worst_bar.get_height()),
    xytext=(0, worst_bar.get_height()+3000),
    ha='center',
    arrowprops=None,
    fontsize=11, color="#C41F1F"
)

plt.tight_layout()
plt.show()

# Dynamic Summary
print(f"📊 Insight: Students sleeping {worst_sleep} hrs show the highest suicidal rate ({worst_rate}%). "
      f"Longer sleep is associated with lower Suicidal Thughts risk.")


In [86]:
df = df[df['Dietary Habits'] != -1]

In [ ]:

# Map dietary habits for clarity
habit_labels = {0:'Poor', 1:'Moderate', 2:'Healthy'}
df['Dietary Habits Label'] = df['Dietary Habits'].map(habit_labels)

# Custom color palette: Poor=Red, Moderate=Yellow, Healthy=Green
habit_colors = ['#e74c3c', '#f1c40f', '#2ecc71']

# Create side-by-side plots
fig, axes = plt.subplots(1, 2, figsize=(14,5))  # 1 row, 2 columns

# Depression vs Dietary Habits
sns.countplot(
    data=df,
    x='Dietary Habits Label',
    hue='Depression',
    palette=['#3498db','#e74c3c'],  # Blue=No depression, Red=Depressed
    order=['Poor','Moderate','Healthy'],
    ax=axes[0]
)
axes[0].set_title(
    "Dietary Habits vs Depression\nPoor diets show higher depression rates",
    fontsize=13, fontweight='bold'
)
axes[0].set_xlabel('Dietary Habits')
axes[0].set_ylabel('Number of Students')
axes[0].legend(title='Depression', labels=['No','Yes'])

# Suicidal Thoughts vs Dietary Habits
sns.countplot(
    data=df,
    x='Dietary Habits Label',
    hue='Suicidal Thoughts',
    palette=['#3498db','#e67e22'],  # Blue=No, Orange=Yes
    order=['Poor','Moderate','Healthy'],
    ax=axes[1]
)
axes[1].set_title(
    "Dietary Habits vs Suicidal Thoughts\nStudents with poor diets report more suicidal thoughts",
    fontsize=13, fontweight='bold'
)
axes[1].set_xlabel('Dietary Habits')
axes[1].set_ylabel('Number of Students')
axes[1].legend(title='Suicidal Thoughts', labels=['No','Yes'])

plt.tight_layout(pad=3)
plt.show()


In [ ]:
# Step 1: Calculate depression counts per stress level
stress_counts = df.groupby('Financial Stress')['Depression'].agg(
    total='count', depressed='sum'
).reset_index()

# Step 2: Calculate depression percentage per level
stress_counts['depr_percent'] = (stress_counts['depressed'] / stress_counts['total'] * 100).round(1)

# Step 3: Sort levels by depression percentage (descending)
sorted_order = stress_counts.sort_values(by='depr_percent', ascending=False)['Financial Stress']

# Step 4: Plot Countplot with hue
plt.figure(figsize=(8,5))
ax = sns.countplot(
    data=df,
    x='Financial Stress',
    hue='Depression',
    palette=['#3498db','#e74c3c'],  # Blue=No, Red=Yes
    order=sorted_order
)

# ✅ Rename Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['No Depression', 'Yes Depression'], title='Depression')

# Step 5: Dynamic Title
worst_stress = stress_counts.sort_values(by='depr_percent', ascending=False).iloc[0]
plt.title(
    f"Financial Stress vs Depression\n"
    f"Highest depression: {worst_stress['Financial Stress']} ({worst_stress['depr_percent']}% depressed)",
    fontsize=13, fontweight='bold'
)
plt.xlabel("Financial Stress Level (0=Low, 2=Moderate, 5=High)")
plt.ylabel("Number of Students")



plt.tight_layout(pad=2)
plt.show()


In [ ]:
# Step 1: Calculate depression counts per stress level
stress_counts = df.groupby('Academic Pressure')['Depression'].agg(
    total='count', depressed='sum'
).reset_index()

# Step 2: Calculate depression percentage per level
stress_counts['depr_percent'] = (stress_counts['depressed'] / stress_counts['total'] * 100).round(1)

# Step 3: Sort levels by depression percentage (descending)
sorted_order = stress_counts.sort_values(by='depr_percent', ascending=False)['Academic Pressure']

# Step 4: Plot Countplot with hue
plt.figure(figsize=(8,5))
ax = sns.countplot(
    data=df,
    x='Academic Pressure',
    hue='Depression',
    palette=['#3498db','#e74c3c'],  # Blue=No, Red=Yes
    order=sorted_order
)

# ✅ Rename Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['No Depression', 'Yes Depression'], title='Depression')

# Step 5: Dynamic Title
worst_stress = stress_counts.sort_values(by='depr_percent', ascending=False).iloc[0]
plt.title(
    f"Academic Pressure vs Depression\n"
    f"Highest depression: {worst_stress['Academic Pressure']} ({worst_stress['depr_percent']}% depressed)",
    fontsize=13, fontweight='bold'
)
plt.xlabel("Academic Pressure Level (1=Low, 3=Moderate, 5=High)")
plt.ylabel("Number of Students")



plt.tight_layout(pad=2)
plt.show()


In [ ]:


# 🎨 Plot aesthetics
sns.set_style("whitegrid")
colors_depr = ['#3498db', '#e74c3c']  # Blue=No, Red=Yes
colors_suic = ['#2ecc71', '#f39c12']  # Green=No, Orange=Yes

fig, axes = plt.subplots(1, 2, figsize=(14,5))  # 1 row, 2 columns

# 1️⃣ Depression vs CGPA
depr_counts = df['Depression'].value_counts(normalize=True)*100
sns.boxplot(
    data=df,
    x='Depression',
    y='CGPA',
    ax=axes[0],
    palette=colors_depr,
    width=0.6
)
axes[0].set_title(
    f"CGPA Distribution vs Depression\n{depr_counts.get(1,0):.1f}% students show Depression",
    fontsize=13, fontweight='bold'
)
axes[0].set_xlabel("Depression (0 = No, 1 = Yes)")
axes[0].set_ylabel("CGPA")

# Annotate mean CGPA
means = df.groupby('Depression')['CGPA'].mean().round(2)
for i, val in enumerate(means):
    axes[0].text(i, val+0.05, f"Mean: {val}", ha='center', color='black', fontweight='bold')

# 2️⃣ Suicidal Thoughts vs CGPA
suic_counts = df['Suicidal Thoughts'].value_counts(normalize=True)*100
sns.boxplot(
    data=df,
    x='Suicidal Thoughts',
    y='CGPA',
    ax=axes[1],
    palette=colors_suic,
    width=0.6
)
axes[1].set_title(
    f"CGPA Distribution vs Suicidal Thoughts\n{suic_counts.get(1,0):.1f}% students reported Suicidal Thoughts",
    fontsize=13, fontweight='bold'
)
axes[1].set_xlabel("Suicidal Thoughts (0 = No, 1 = Yes)")
axes[1].set_ylabel("CGPA")

# Annotate mean CGPA
means_suic = df.groupby('Suicidal Thoughts')['CGPA'].mean().round(2)
for i, val in enumerate(means_suic):
    axes[1].text(i, val+0.05, f"Mean: {val}", ha='center', color='black', fontweight='bold')

plt.tight_layout(pad=2)
plt.show()


In [ ]:


sns.set_style("whitegrid")
fig, axes = plt.subplots(2, 1, figsize=(12, 10))  # 2 rows for 2 insights

# ================================
# 1️⃣ Depression %
# ================================
depr_data = df.groupby(['Sleep Duration','Dietary Habits'])['Depression'].mean().reset_index()
depr_data['Depression %'] = (depr_data['Depression']*100).round(1)

ax1 = sns.barplot(
    data=depr_data,
    x='Sleep Duration',
    y='Depression %',
    hue='Dietary Habits',
    palette='coolwarm',
    ax=axes[0]
)

# Annotate percentages
for p in ax1.patches:
    ax1.annotate(
        f"{p.get_height():.1f}%",
        (p.get_x() + p.get_width()/2, p.get_height()),
        ha='center', va='bottom', fontsize=9, color='black', fontweight='bold'
    )

axes[0].set_title(
    "How Sleep + Dietary Habits Affect Depression\n",
    fontsize=13, fontweight='bold'
)
axes[0].set_xlabel("Sleep Duration (hrs)")
axes[0].set_ylabel("Depression Rate (%)")
axes[0].legend(title="Dietary Habits\n0=Poor, 1=Moderate, 2=Healthy")

# ================================
# 2️⃣ Suicidal Thoughts %
# ================================
suic_data = df.groupby(['Sleep Duration','Dietary Habits'])['Suicidal Thoughts'].mean().reset_index()
suic_data['Suicidal %'] = (suic_data['Suicidal Thoughts']*100).round(1)

ax2 = sns.barplot(
    data=suic_data,
    x='Sleep Duration',
    y='Suicidal %',
    hue='Dietary Habits',
    palette='mako',
    ax=axes[1]
)

# Annotate percentages
for p in ax2.patches:
    ax2.annotate(
        f"{p.get_height():.1f}%",
        (p.get_x() + p.get_width()/2, p.get_height()),
        ha='center', va='bottom', fontsize=9, color='black', fontweight='bold'
    )

axes[1].set_title(
    "How Sleep + Dietary Habits Affect Suicidal Thoughts\n",
    fontsize=13, fontweight='bold'
)
axes[1].set_xlabel("Sleep Duration (hrs)")
axes[1].set_ylabel("Suicidal Thoughts Rate (%)")
axes[1].legend(title="Dietary Habits\n0=Poor, 1=Moderate, 2=Healthy")

plt.tight_layout(pad=3)
plt.show()


In [129]:
# ...existing code...
df = df[df['Academic Pressure'] != 0]
# ...existing code...

In [ ]:
# Combine Academic + Financial Stress Impact on Depression
stress_combo = (
    df.groupby(['Academic Pressure', 'Financial Stress'])['Depression']
    .agg(['count', 'sum'])
    .reset_index()
    .rename(columns={'count': 'Total', 'sum': 'Depressed'})
)
stress_combo['Depression%'] = (stress_combo['Depressed'] / stress_combo['Total'] * 100).round(1)

# Sort by depression percentage
stress_combo = stress_combo.sort_values('Depression%', ascending=False)

# Plot
plt.figure(figsize=(12,6))
ax = sns.barplot(
    data=stress_combo,
    x='Academic Pressure',
    y='Depression%',
    hue='Financial Stress',
    palette='coolwarm'
)

plt.title('Depression % by Academic & Financial Stress Levels', fontsize=14, fontweight='bold')
plt.xlabel('Academic Pressure (1=Low, 5=High)')
plt.ylabel('Depression %')
plt.legend(title='Financial Stress (1=Low, 5=High)')

# Annotate bars
for p in ax.patches:
    ax.annotate(f'{p.get_height()}%', 
                (p.get_x() + p.get_width()/2, p.get_height()), 
                ha='center', va='bottom', fontsize=9, color='black', rotation=0)

plt.tight_layout()
plt.show()


In [ ]:


plt.figure(figsize=(18,5))

# 1️⃣ CGPA vs Depression
plt.subplot(1,3,1)
sns.histplot(data=df, x='CGPA', hue='Depression', kde=True, element='step', palette=['#3498db','#e74c3c'])
plt.title('CGPA Distribution by Depression\n(Blue=No, Red=Yes)', fontsize=12, fontweight='bold')
plt.xlabel('CGPA')
plt.ylabel('Number of Students')

# 2️⃣ Study Hours vs Depression
plt.subplot(1,3,2)
sns.histplot(data=df, x='Study Hours', hue='Depression', kde=True, element='step', palette=['#3498db','#e74c3c'])
plt.title('Study Hours Distribution by Depression\n(Blue=No, Red=Yes)', fontsize=12, fontweight='bold')
plt.xlabel('Study Hours')
plt.ylabel('Number of Students')

# 3️⃣ Sleep Hours vs Depression
plt.subplot(1,3,3)
sns.histplot(data=df, x='Sleep Duration', hue='Depression', kde=True, element='step', palette=['#3498db','#e74c3c'])
plt.title('Sleep Hours Distribution by Depression\n(Blue=No, Red=Yes)', fontsize=12, fontweight='bold')
plt.xlabel('Sleep Duration (hrs)')
plt.ylabel('Number of Students')

plt.tight_layout()
plt.show()


In [ ]:
# Select numeric columns for correlation
numeric_df = df.select_dtypes(include=['int64', 'float64'])

plt.figure(figsize=(10,7))
sns.heatmap(
    numeric_df.corr(), 
    annot=True, fmt=".2f", cmap='coolwarm', 
    cbar=True, square=True
)
plt.title('Correlation Heatmap of All Numeric Features', fontsize=14, fontweight='bold')
plt.show()
